In [ ]:
# Setup before running notebook:
# In Config.py
#    IK_SOLVER == 'trac_ik' if you want to use trac_ik
# In DomainConfig.py:
#    ROBOT_NAME = 'fetch'
#    OPENRAVE_ENV_XML = 'path to any env dae file used for fetch'

In [ ]:
import os, sys
sys.path.append('../../')

In [ ]:
import Config
from src.Simulators.OpenRaveSimulator import *
from openravepy import *
from tf.transformations import *
from src.OpenraveUtils import *
from prpy.planning import ompl, CBiRRTPlanner, mk

In [ ]:
# Load environment
sim = OpenRaveSimulator(Config.OPENRAVE_ENV_XML)
env = sim.env
robot = env.GetRobot(Config.ROBOT_NAME)

In [ ]:
# Go to pose
quat = [1,0,0,0]
trans = [0.2,0.4,0.7]
# trans = [0.,0.,0.]
goal_transform_values = matrixFromPose(quat+trans)

In [ ]:
# Fetch works with  both ikfast and trac_ik
# iksolutions = sim.get_trac_ik_solutions(robot, goal_transform_values, check_collisions=True)
iksolutions = sim.get_ik_solutions(robot, goal_transform_values, check_collisions=False)
robot.SetActiveDOFValues(iksolutions[0])

In [ ]:
# Delta trajectories with CBiRRT 
planner = CBiRRTPlanner(timelimit=10.)
direction = [0.01,1.,0.]
distance = 1
trajectory_object = planner.PlanToEndEffectorOffset(robot, direction, distance,smoothingitrs=10)
res = planningutils.RetimeTrajectory(trajectory_object)
trajectory_object = trajectory_object.serialize()

In [ ]:
# Execute Trajectory
traj = RaveCreateTrajectory(env, '')
Trajectory.deserialize(traj, trajectory_object)
with robot:
    robot.GetController().SetPath(traj)
robot.WaitForController(0)